In [1]:
import random
import json
import string
import pickle
import emoji
import re
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import OneHotEncoder

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [2]:
lemmatizer = WordNetLemmatizer()

def get_json_data(file):
    return json.loads(open(file).read())

def clean_text(text: str) -> str:
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = emoji.demojize(text)
    text = re.sub(r"\:(.*?)\:", '', text)
    text = re.sub(r"\s+", " ", text)
    text = ''.join([lemmatizer.lemmatize(word) for word in text])
    return text

In [3]:
lemmetizer = WordNetLemmatizer()

intents  = get_json_data('basic_intents.json')
training_intents = get_json_data('training_intents.json')

intents.extend(training_intents)

In [4]:
words = []
X = []
y = []

for intent in intents:
    for pattern in intent['patterns']:
        cleaned_pattern = clean_text(pattern)
        word_list = word_tokenize(cleaned_pattern)
        words.extend(word_list)
        X.append(word_list)
        y.append(intent['tag'])


words = sorted(set(words))

In [5]:
pickle.dump(words, open('all_words.pickle', 'wb'))

In [6]:
train_X = []

for idx, doc in enumerate(X):
    bag = []

    for word in words:
        bag.append(1) if word in doc else bag.append(0)
    train_X.append(bag)


In [7]:
ohe = OneHotEncoder(sparse=False)

y_encoded = np.array(y).reshape(-1, 1)

y_encoded = ohe.fit_transform(y_encoded).tolist()


In [8]:
train_data = list(zip(train_X, y_encoded))

random.shuffle(train_data)

train_X, y_encoded = zip(*train_data)

In [9]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_X[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_encoded[0]), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-05-17 13:59:53.514896: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-17 13:59:53.515029: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
history = model.fit(train_X, y_encoded, epochs=50, batch_size=5, verbose=1)

Epoch 1/50


2022-05-17 13:59:53.773972: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-05-17 13:59:53.936790: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


22/22 [==============================] - 0s 8ms/step - loss: 2.6063 - accuracy: 0.0727
Epoch 2/50
22/22 [==============================] - 0s 8ms/step - loss: 2.4859 - accuracy: 0.1727
Epoch 3/50
22/22 [==============================] - 0s 8ms/step - loss: 2.4013 - accuracy: 0.2091
Epoch 4/50
22/22 [==============================] - 0s 9ms/step - loss: 2.2402 - accuracy: 0.3364
Epoch 5/50
22/22 [==============================] - 0s 9ms/step - loss: 2.0700 - accuracy: 0.3909
Epoch 6/50
22/22 [==============================] - 0s 8ms/step - loss: 1.9592 - accuracy: 0.4364
Epoch 7/50
22/22 [==============================] - 0s 8ms/step - loss: 1.7744 - accuracy: 0.5364
Epoch 8/50
22/22 [==============================] - 0s 7ms/step - loss: 1.5785 - accuracy: 0.5909
Epoch 9/50
22/22 [==============================] - 0s 9ms/step - loss: 1.3827 - accuracy: 0.7182
Epoch 10/50
22/22 [==============================] - 0s 8ms/step - loss: 1.2712 - accuracy: 0.7182
Epoch 11/50
22/22 [===========

In [11]:
try:
    model.save('chatbot_model.h5', history)
    print('Sucesfully saved the model!')
except Exception as e:
    print(f'Error while saving the model: {str(e)}')

Sucesfully saved the model!
